In [1]:
# importing libraries
import pandas as pd
import warnings
import os
from google.cloud import bigquery
import pandas as pd
from tqdm import tqdm
import numpy as np
import datetime
from datetime import date
import re
import json
import tracemalloc
from google.cloud import storage
import pandasql as ps
import warnings
import Levenshtein as lev
import haversine as hs

In [2]:
warnings.simplefilter('ignore')
# adding bigquery json key to the os GOOGLE_APPLICATION_CREDENTIALS
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/akivirus/Desktop/Project/BQ_queries/keys/key.json"

In [3]:
print("Fetching Data from BigQuery Table.")
bq_client = bigquery.Client()
query = """

# Select Phone
#   FROM `Zoho_CRM_Cleanup.ds_temp_enriched_leads_28Apr23` Z
#   Where Z.State is null or Z.State like ''

Select Phone
  FROM `Zoho_CRM_Cleanup.ds_temp_enriched_leads_28Apr23` Z
  Where Z.State is not null

# with comp4 as (
#   select substr(cast(phone_number as string), 3) as phone_num from `merapashu360-buyer-app.ds_competitor_4.ds_comp4_leads_fulldump_v2`
# ),

# contact_details as (
#   select phone_num from `merapashu360-buyer-app.ds_customer_analytics.ds_contacts_location_via_phonenum`
# )

# select 
# distinct * 
# from 
# (select * from comp4
# except distinct
# select * from contact_details)

"""
data = bq_client.query(query).result()
data_df = pd.DataFrame(data)
data_list = []
for i in tqdm(range(len(data_df))):
    data_list.append(list(data_df.iloc[i,0]))
col_headers = [field.name for field in data.schema]
df_bq = pd.DataFrame(data_list, columns=col_headers)


Fetching Data from BigQuery Table.


100%|██████████| 198026/198026 [00:02<00:00, 94799.31it/s]


In [4]:
df_bq.columns = ['phone_num']
df_bq.to_csv( './csv_data/zoho_crm_data_cleanup_validate.csv',index=False)